In [3]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing import text,sequence
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN,LSTM,SpatialDropout1D,GRU,Bidirectional,Input,Dense,Activation,Dropout
# from keras.layers.core import Dense#,Activation,Dropout

ImportError: cannot import name 'layers' from partially initialized module 'keras.src' (most likely due to a circular import) (C:\Users\Richa\OneDrive\Documents\Desktop\kernel_jupyter\mynewv\lib\site-packages\keras\src\__init__.py)

In [ ]:
from tqdm import tqdm

In [ ]:
#configurint TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    #default distribution strategy in tensorflow, Works on CPU and single GPU
    strategy = tf.distribute.OneDeviceStrategy("CPU:0")

In [ ]:
train = pd.read_csv('C:/Users/Richa/OneDrive/Documents/Desktop/TARP PROJECT_TOXIC COMMENT/dataset/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('C:/Users/Richa/OneDrive/Documents/Desktop/TARP PROJECT_TOXIC COMMENT/dataset/validation.csv')
test = pd.read_csv('C:/Users/Richa/OneDrive/Documents/Desktop/TARP PROJECT_TOXIC COMMENT/dataset/test.csv')

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis = 1, inplace = True)

In [ ]:
#check max len of comment_text column to use this for padding in future
pad_len = train['comment_text'].apply(lambda x:len(str(x).split())).max()
print('max len of comment_text column',pad_len)

DATA PREPARATION

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, stratify = train.toxic.values, random_state = 42,test_size = 0.3,shuffle = True)

In [ ]:
len(xtrain),len(xvalid)

### Tokenisation and Padding with max len of words in curpus

In [ ]:
test.head()

In [ ]:
#using keras tokenizer
token = text.Tokenizer(num_words = None)
max_len = 2400
xtest = test.content.values
token.fit_on_texts(list(xtrain) + list(xvalid) + list(xtest))

x_train_seq = token.texts_to_sequences(xtrain)
x_valid_seq = token.texts_to_sequences(xvalid)
x_test_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
x_train_pad = pad_sequences(x_train_seq,maxlen = max_len)


x_valid_pad = pad_sequences(x_valid_seq,maxlen = max_len)
x_test_pad = pad_sequences(x_test_seq,maxlen = max_len)
word_index = token.word_index

### Classification based on GRU(Gated Recurrent Unit)

In [ ]:
model.fit(x_train_pad,ytrain, epochs = 1,batch_size = 128*strategy.num_replicas_in_sync)

In [ ]:
gru_pred = model.predict(x_valid_pad)

In [ ]:
model_accuracy = roc_auc_score(yvalid,gru_pred)
model_accuracy_ls.append({'model':'GRU','AUC_SCORE':model_accuracy})

In [ ]:
model_accuracy_ls